In [1]:
!pip install gym==0.26.2 --upgrade --quiet



In [4]:
!pip install numpy --upgrade --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 84.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dopamine-rl 4.1.2 requires gym<=0.25.2, but you have gym 0.26.2 which is incompatible.
cupy-cuda12x 13.3.0 requires numpy<2.3,>=1.22, but you have numpy 2.3.1 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.3.1 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.3.1 which is incompatible.


In [3]:
!pip install numpy==1.26.4 --force-reinstall --quiet



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 69.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dopamine-rl 4.1.2 requires gym<=0.25.2, but you have gym 0.26.2 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [1]:
#  Install if not done already
!pip install snntorch --quiet


#  Imports
import torch
import torch.nn as nn
import snntorch as snn
import snntorch.functional as SF
import gym
import numpy as np
import random

#  Environment with new API enabled
env = gym.make("FrozenLake-v1", is_slippery=False, render_mode=None)

n_states = env.observation_space.n
n_actions = env.action_space.n

#  Q-table
q_table = np.zeros((n_states, n_actions))

#  Hyperparameters
episodes = 200
learning_rate = 0.1
gamma = 0.99
epsilon = 1.0
epsilon_decay = 0.995
min_epsilon = 0.01
beta = 0.95  # for LIF neurons

#  SNN Encoder
class StateEncoder(nn.Module):
    def __init__(self, n_states, hidden_size):
        super().__init__()
        self.fc = nn.Linear(n_states, hidden_size)
        self.lif = snn.Leaky(beta=beta)

    def forward(self, x):
        cur = self.fc(x)
        spk, _ = self.lif(cur)
        return spk

encoder = StateEncoder(n_states=n_states, hidden_size=16)

#  STDP-like weight update
stdp_weights = torch.rand((n_states, 16), requires_grad=False)

def stdp_update(pre_spikes, post_spikes, reward, lr=0.01):
    dw = torch.outer(pre_spikes, post_spikes)
    return reward * dw * lr

#  One-hot encode state
def one_hot(state, n_states):
    x = torch.zeros(n_states)
    x[state] = 1.0
    return x

#  Training loop with new step/reset API
for ep in range(episodes):
    state, info = env.reset()  # returns (observation, info)

    done = False

    while not done:
        state_tensor = one_hot(state, n_states)
        encoded_spikes = encoder(state_tensor)

        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample()
        else:
            action = np.argmax(q_table[state])

        next_state, reward, terminated, truncated, info = env.step(action)
        done = terminated or truncated

        # Q-learning update
        q_table[state, action] += learning_rate * (
            reward + gamma * np.max(q_table[next_state]) - q_table[state, action]
        )

        # STDP update
        pre = state_tensor.detach()
        post = encoded_spikes.detach()
        stdp_weights.data += stdp_update(pre, post, reward)

        state = next_state

    epsilon = max(min_epsilon, epsilon * epsilon_decay)

print("\n Training complete!")
print(f"Final epsilon: {epsilon:.3f}")


/usr/local/lib/python3.11/dist-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):



 Training complete!
Final epsilon: 0.367
